In [16]:
# 基本模块
import requests
import pandas as pd
from requests_html import HTMLSession
import time
from random import random

In [2]:
# A-1   单一页面
url = "https://www.liepin.com/zhaopin/?keyword=数据分析"
session = HTMLSession()
r = session.get( url )

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//ul[@class="sojob-list"]/li')

# 预期是一个元素的列表？
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]'))
#print (主要元素[0].xpath('//div[contains(@class,"job-info")]/h3/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="text-warning"]'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/time/@title'))
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a'))

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
        '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
        '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
        '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
        '职称':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司地点': '//div[contains(@class,"job-info")]/p/a',
        '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    },
    'text_content': {
    },
    'href': {
        '链结':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

[len(v) for k,v in 数据字典.items()]

数据 = pd.DataFrame(数据字典)
#数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
数据 

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL
0,大专及以上,10年以上,25-35k·12薪,2020年04月26日,人防设计工程师,广州-黄埔区,广州嘉殷建设管理有限公司,https://www.liepin.com/job/1927791699.shtml,https://www.liepin.com/company/12195615/
1,大专及以上,3-5年,6-8k·12薪,2020年04月26日,测试工程师,西安-未央区,星展测控科技股份有限公司,https://www.liepin.com/job/1927789815.shtml,https://www.liepin.com/company/10158561/
2,学历不限,1-3年,4-6k·12薪,2020年04月26日,装配技工,西安-未央区,星展测控科技股份有限公司,https://www.liepin.com/job/1927789491.shtml,https://www.liepin.com/company/10158561/
3,大专及以上,1-3年,面议,2020年04月26日,客户经理（汽车融资租赁）,南昌,创富融资租赁(上海)有限公司,https://www.liepin.com/job/1927788671.shtml,https://www.liepin.com/company/7854759/
4,本科及以上,3-5年,16-25k·12薪,2020年04月26日,golang开发工程师,成都-高新区,成都零点能量信息技术有限责任公司,https://www.liepin.com/job/1927787679.shtml,https://www.liepin.com/company/9326766/
5,大专及以上,5-10年,15-20k·12薪,2020年04月26日,成本工程师（土建）,承德,承德天缘盛世房地产开发有限公司,https://www.liepin.com/job/1927785791.shtml,https://www.liepin.com/company/9143037/
6,大专及以上,3-5年,8-16k·12薪,2020年04月26日,渠道经理,苏州,苏州浩辰软件股份有限公司,https://www.liepin.com/job/1927784455.shtml,https://www.liepin.com/company/7965554/
7,大专及以上,5-10年,面议,2020年04月26日,区域销售经理,上海,上海迪罗电气有限公司,https://www.liepin.com/job/1927784195.shtml,https://www.liepin.com/company/7927458/
8,本科及以上,经验不限,面议,2020年04月26日,国际保险理赔总监,大连,维特奥健康,https://www.liepin.com/job/1927783939.shtml,https://www.liepin.com/company/8375949/
9,大专及以上,1-3年,5-7k·12薪,2020年04月26日,出纳,深圳-龙华区,北京斯雷康科技股份有限公司,https://www.liepin.com/job/1927783881.shtml,https://www.liepin.com/company/1408548/


# 公司

In [55]:
# A-2 扩张 公司?  

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath('//div[@data-selector="search-conditions"]')
# 预期是一个元素的列表？
print (主要元素)
print (主要元素[0])
print (主要元素[0].xpath('//dt[@class="search-title"]'))

list_search_title = 主要元素[0].xpath('//dt[@class="search-title"]')
for x in list_search_title:
    print (x.text)
    
list_search_dd = 主要元素[0].xpath('//dt[@class="search-title"]/following-sibling::dd')
for x in list_search_dd:
    print (x)  
    

公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a/@href')
               
公司数据选择器链结

# 但我们需要知道这些选择器链结, 对映到什麽数据
公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a')
公司数据选择器链结

#[ x.xpath("a/@href")[0] for x in 公司数据选择器链结]
#[ x.xpath("a/text()")[0] for x in 公司数据选择器链结]
公司数据选择器链结 = { x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in 公司数据选择器链结}
公司数据选择器链结

[<Element 'div' class=('search-conditions',) data-selector='search-conditions'>]
<Element 'div' class=('search-conditions',) data-selector='search-conditions'>
[<Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>]
公司：
行业：
城市：
薪资：
更多：
<Element 'dd' class=('comp-list',)>
<Element 'dd' class=('short-dd', 'select-industry') data-param='industries'>
<Element 'dd' data-param='city'>
<Element 'dd' data-param='salary'>
<Element 'dd' class=('dropdown', 'dropdown-time')>
<Element 'dd' class=('dropdown', 'dropdown-jobkind')>
<Element 'dd' class=('dropdown', 'dropdown-compscale')>
<Element 'dd' class=('dropdown', 'dropdown-compkind')>


{'中国500强': '/zhaopin/?init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&compTag=155&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_pageSize=40&d_headId=41d615212789e37ab84f0440a67481f7',
 '2018互联网300强': '/zhaopin/?init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&compTag=182&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_pageSize=40&d_headId=41d615212789e37ab84f0440a67481f7',
 '制造业500强': '/zhaopin/?init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&compTag=186&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_

In [56]:
from urllib.parse import urlparse, parse_qs
[ urlparse(x) for x in 公司数据选择器链结.values()]

[ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&compTag=155&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_pageSize=40&d_headId=41d615212789e37ab84f0440a67481f7', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&compTag=182&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_pageSize=40&d_headId=41d615212789e37ab84f0440a67481f7', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E

In [57]:
import pandas as pd
df = pd.DataFrame([ urlparse(x) for x in 公司数据选择器链结.values()])
df.info()
print(df.nunique())
df.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   scheme    6 non-null      object
 1   netloc    6 non-null      object
 2   path      6 non-null      object
 3   params    6 non-null      object
 4   query     6 non-null      object
 5   fragment  6 non-null      object
dtypes: object(6)
memory usage: 416.0+ bytes
scheme      1
netloc      1
path        1
params      1
query       6
fragment    1
dtype: int64


,scheme,netloc,path,params,query,fragment
0,,,/zhaopin/,,init=-1&headckid=0827198a456f77be&flushckid=1&...,


In [58]:
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print (df_qs.nunique())
df_qs.head()
df_qs[['keyword','compTag']]

init             1
headckid         1
flushckid        1
fromSearchBtn    1
keyword          1
compTag          6
ckid             1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
dtype: int64


,keyword,compTag
0,数据分析,155
1,数据分析,182
2,数据分析,186
3,数据分析,189
4,数据分析,130
5,数据分析,156


In [59]:
# 列表暂存
列表暫存 = [] # list()
for q in df['query']: 
    字典暫存 = dict()
    for k,v in parse_qs(q).items(): # for 鍵,值 in 字典.items():
        字典暫存.update({k:v[0]})
    列表暫存.append(字典暫存)
列表暫存

[{'init': '-1',
  'headckid': '0827198a456f77be',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'keyword': '数据分析',
  'compTag': '155',
  'ckid': '0827198a456f77be',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw',
  'd_sfrom': 'search_unknown',
  'd_ckId': '41d615212789e37ab84f0440a67481f7',
  'd_curPage': '0',
  'd_pageSize': '40',
  'd_headId': '41d615212789e37ab84f0440a67481f7'},
 {'init': '-1',
  'headckid': '0827198a456f77be',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'keyword': '数据分析',
  'compTag': '182',
  'ckid': '0827198a456f77be',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw',
  'd_sfrom': 'search_unknown',
  'd_ckId': '41d615212789e37ab84f0440a67481f7',
  'd_curPage': '0',
  'd_pageSize': '40',
  'd_headId': '41d615212789e37ab84f0440a67481f7'},
 {'init': '-1',
  'headckid': '0827198a456f77be',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'keyword': '数据分析',
  'compTag': '186',
  'ckid': '0827198a456f77be',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXq

In [60]:
# B-4 建构 参数模板 及 字典_compTag
def parse_url_qs_for_compTag (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])['compTag']
参数模板 = parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])
print(参数模板)
# [ parse_url_qs_for_compTag(x)['compTag'] for x in 公司数据选择器链结.values()]
[ parse_url_qs_for_compTag(x)['compTag'][0] for x in 公司数据选择器链结.values()]

字典_compTag = { k:parse_url_qs_for_compTag(v)['compTag'][0] for k,v in 公司数据选择器链结.items()}
print (字典_compTag)

{'init': ['-1'], 'headckid': ['0827198a456f77be'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['数据分析'], 'compTag': ['155'], 'ckid': ['0827198a456f77be'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['41d615212789e37ab84f0440a67481f7'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['41d615212789e37ab84f0440a67481f7']}
{'中国500强': '155', '2018互联网300强': '182', '制造业500强': '186', 'AI创新成长50强 ': '189', '独角兽': '130', '上市公司': '156'}


# 行业

In [61]:
# A-2 扩张 行业 ?  

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath('//div[@data-selector="search-conditions"]')
# 预期是一个元素的列表？
print (主要元素)
print (主要元素[0])
print (主要元素[0].xpath('//dt[@class="search-title"]'))

list_search_title = 主要元素[0].xpath('//dt[@class="search-title"]')
for x in list_search_title:
    print (x.text)
    
list_search_dd = 主要元素[0].xpath('//dt[@class="search-title"]/following-sibling::dd')
for x in list_search_dd:
    print (x)  
    

行业数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[1] \
                    .xpath('//div[contains(@class,"sub-industry")]/a/@href')
               
行业数据选择器链结

# 但我们需要知道这些选择器链结, 对映到什麽数据
行业数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[1] \
                    .xpath('//div[contains(@class,"sub-industry")]/a')
行业数据选择器链结

#[ x.xpath("a/@href")[0] for x in 公司数据选择器链结]
#[ x.xpath("a/text()")[0] for x in 公司数据选择器链结]
行业数据选择器链结 = { x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in 行业数据选择器链结}
行业数据选择器链结

[<Element 'div' class=('search-conditions',) data-selector='search-conditions'>]
<Element 'div' class=('search-conditions',) data-selector='search-conditions'>
[<Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>]
公司：
行业：
城市：
薪资：
更多：
<Element 'dd' class=('comp-list',)>
<Element 'dd' class=('short-dd', 'select-industry') data-param='industries'>
<Element 'dd' data-param='city'>
<Element 'dd' data-param='salary'>
<Element 'dd' class=('dropdown', 'dropdown-time')>
<Element 'dd' class=('dropdown', 'dropdown-jobkind')>
<Element 'dd' class=('dropdown', 'dropdown-compscale')>
<Element 'dd' class=('dropdown', 'dropdown-compkind')>


{'互联网/电商': '/zhaopin/?subIndustry=&init=-1&industryType=industry_01&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&industries=040&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_pageSize=40&d_headId=41d615212789e37ab84f0440a67481f7',
 '游戏产业': '/zhaopin/?subIndustry=&init=-1&industryType=industry_01&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&industries=420&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_pageSize=40&d_headId=41d615212789e37ab84f0440a67481f7',
 '计算机软件': '/zhaopin/?subIndustry=&init=-1&industryType=industry_01&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&industries=010&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=0827198a456f77be&siTag=1B2M2Y8AsgT

In [62]:
from urllib.parse import urlparse, parse_qs
[ urlparse(x) for x in 行业数据选择器链结.values()]
import pandas as pd
df = pd.DataFrame([ urlparse(x) for x in 行业数据选择器链结.values()])
df.info()
print(df.nunique())
df.head(1)
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print (df_qs.nunique())
df_qs.head()
df_qs[['keyword','industryType','industries']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   scheme    51 non-null     object
 1   netloc    51 non-null     object
 2   path      51 non-null     object
 3   params    51 non-null     object
 4   query     51 non-null     object
 5   fragment  51 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB
scheme       1
netloc       1
path         1
params       1
query       51
fragment     1
dtype: int64
init              1
industryType     12
headckid          1
flushckid         1
fromSearchBtn     1
industries       51
keyword           1
ckid              1
siTag             1
d_sfrom           1
d_ckId            1
d_curPage         1
d_pageSize        1
d_headId          1
dtype: int64


,keyword,industryType,industries
0,数据分析,industry_01,040
1,数据分析,industry_01,420
2,数据分析,industry_01,010
3,数据分析,industry_01,030
4,数据分析,industry_02,050
5,数据分析,industry_02,060
6,数据分析,industry_02,020
7,数据分析,industry_03,080
8,数据分析,industry_03,100
9,数据分析,industry_03,090


In [63]:
# B-4 建构 参数模板 及 字典_compTag
def parse_url_qs_for_compTag (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])['compTag']
参数模板 = parse_url_qs_for_compTag(list(行业数据选择器链结.values())[0])
print(参数模板)
# [ parse_url_qs_for_compTag(x)['compTag'] for x in 公司数据选择器链结.values()]
[ parse_url_qs_for_compTag(x)['industries'][0] for x in 行业数据选择器链结.values()]

字典_industries = { k:parse_url_qs_for_compTag(v)['industries'][0] for k,v in 行业数据选择器链结.items()}
print (字典_industries)

{'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['0827198a456f77be'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['040'], 'keyword': ['数据分析'], 'ckid': ['0827198a456f77be'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['41d615212789e37ab84f0440a67481f7'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['41d615212789e37ab84f0440a67481f7']}
{'互联网/电商': '040', '游戏产业': '420', '计算机软件': '010', 'IT服务': '030', '电子/芯片/半导体': '050', '通信业': '060', '计算机/网络设备': '020', '房地产/建筑': '080', '规划/设计/装潢': '100', '房地产服务': '090', '银行': '130', '保险': '140', '基金/证券/投资': '150', '会计/审计': '430', '信托/担保/拍卖': '500', '快消品': '190', '批发零售': '240', '服装纺织': '200', '家具/家电': '210', '办公设备': '220', '奢侈品/收藏品': '460', '珠宝/玩具/工艺品': '470', '汽车/摩托车': '350', '机械/机电/重工': '360', '印刷/包装/造纸': '180', '原材料加工': '370', '仪器/电气/自动化': '340', '制药/生物工程': '270', '医疗/保健/美容': '280', '医疗器械': '290', '能源/水利': '330', '石油/化工': '310', '采掘/冶炼/矿产': '320', '环保': '300'

In [64]:
# B-5 建构 参数模板  
def 参数模板生成(industries , keyword ):
    参数 = 参数模板.copy()
    参数['industries'] = industries
    参数['keyword'] = keyword
    return (参数)

参数_industries_数据分析 = { k:参数模板生成(industries = [v], keyword = ['数据分析']) for k,v in 字典_industries.items()}
print(参数_industries_数据分析)

{'互联网/电商': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['0827198a456f77be'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['040'], 'keyword': ['数据分析'], 'ckid': ['0827198a456f77be'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['41d615212789e37ab84f0440a67481f7'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['41d615212789e37ab84f0440a67481f7']}, '游戏产业': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['0827198a456f77be'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['420'], 'keyword': ['数据分析'], 'ckid': ['0827198a456f77be'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['41d615212789e37ab84f0440a67481f7'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['41d615212789e37ab84f0440a67481f7']}, '计算机软件': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['0827198a456f77be'], 'flushckid': ['1'], 

# 城市

In [65]:
# A-2 扩张 城市?  

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath('//div[@data-selector="search-conditions"]')
# 预期是一个元素的列表？
print (主要元素)
print (主要元素[0])
print (主要元素[0].xpath('//dt[@class="search-title"]'))

list_search_title = 主要元素[0].xpath('//dt[@class="search-title"]')
for x in list_search_title:
    print (x.text)
    
list_search_dd = 主要元素[0].xpath('//dt[@class="search-title"]/following-sibling::dd')
for x in list_search_dd:
    print (x)  
    

城市数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[2] \
                    .xpath('//a/@href')
               
城市数据选择器链结

# 但我们需要知道这些选择器链结, 对映到什麽数据
城市数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[2] \
                    .xpath('//a')
城市数据选择器链结

#[ x.xpath("a/@href")[0] for x in 公司数据选择器链结]
#[ x.xpath("a/text()")[0] for x in 公司数据选择器链结]
城市数据选择器链结 = { x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in 城市数据选择器链结}
城市数据选择器链结

[<Element 'div' class=('search-conditions',) data-selector='search-conditions'>]
<Element 'div' class=('search-conditions',) data-selector='search-conditions'>
[<Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>]
公司：
行业：
城市：
薪资：
更多：
<Element 'dd' class=('comp-list',)>
<Element 'dd' class=('short-dd', 'select-industry') data-param='industries'>
<Element 'dd' data-param='city'>
<Element 'dd' data-param='salary'>
<Element 'dd' class=('dropdown', 'dropdown-time')>
<Element 'dd' class=('dropdown', 'dropdown-jobkind')>
<Element 'dd' class=('dropdown', 'dropdown-compscale')>
<Element 'dd' class=('dropdown', 'dropdown-compkind')>


{'全国': '/zhaopin/?init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&dqs=&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_pageSize=40&d_headId=41d615212789e37ab84f0440a67481f7',
 '北京': '/zhaopin/?init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&dqs=010&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_pageSize=40&d_headId=41d615212789e37ab84f0440a67481f7',
 '上海': '/zhaopin/?init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&dqs=020&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_pageSize=40&d_headId=41d615212789

In [66]:
from urllib.parse import urlparse, parse_qs
[ urlparse(x) for x in 城市数据选择器链结.values()]


[ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&dqs=&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_pageSize=40&d_headId=41d615212789e37ab84f0440a67481f7', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&dqs=010&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=0827198a456f77be&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=41d615212789e37ab84f0440a67481f7&d_curPage=0&d_pageSize=40&d_headId=41d615212789e37ab84f0440a67481f7', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=0827198a456f77be&flushckid=1&fromSearchBtn=2&dqs=020&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90

In [67]:
import pandas as pd
df = pd.DataFrame([ urlparse(x) for x in 行业数据选择器链结.values()])
df.info()
print(df.nunique())
df.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   scheme    51 non-null     object
 1   netloc    51 non-null     object
 2   path      51 non-null     object
 3   params    51 non-null     object
 4   query     51 non-null     object
 5   fragment  51 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB
scheme       1
netloc       1
path         1
params       1
query       51
fragment     1
dtype: int64


,scheme,netloc,path,params,query,fragment
0,,,/zhaopin/,,subIndustry=&init=-1&industryType=industry_01&...,


In [68]:
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print (df_qs.nunique())
df_qs.head()
df_qs[['keyword','headckid']]

init              1
industryType     12
headckid          1
flushckid         1
fromSearchBtn     1
industries       51
keyword           1
ckid              1
siTag             1
d_sfrom           1
d_ckId            1
d_curPage         1
d_pageSize        1
d_headId          1
dtype: int64


,keyword,headckid
0,数据分析,0827198a456f77be
1,数据分析,0827198a456f77be
2,数据分析,0827198a456f77be
3,数据分析,0827198a456f77be
4,数据分析,0827198a456f77be
5,数据分析,0827198a456f77be
6,数据分析,0827198a456f77be
7,数据分析,0827198a456f77be
8,数据分析,0827198a456f77be
9,数据分析,0827198a456f77be


In [69]:
# B-3-X 对 B-3 代码的字典/列表推导的分拆说明

# ----------------------------------------------
# 列表暂存 = [{k:v[0] for k,v in parse_qs(q).items()} for q in df['query'] ]
# 以下3行代碼相當於上面推導1行
列表暂存 = [] # list()
for x in df['query']: 
    列表暂存.append({k:v[0] for k,v in parse_qs(x).items()} )
    
# ----------------------------------------------
# 字典暂存 = {k:v[0] for k,v in parse_qs(x).items()}
# 以下3行代碼相當於上面推導1行
字典暂存 = dict()
for k,v in parse_qs(x).items():     # for 键,值 in 字典.items():
    字典暂存.update({k:v[0]})
    
# ----------------------------------------------
print (列表暂存, 字典暂存) 

'''python 原代碼以下不處理
df_qs = pd.DataFrame(list_query)
print (df_qs.nunique())
df_qs.head()
df_qs[['keyword','compTag']]
'''

[{'init': '-1', 'industryType': 'industry_01', 'headckid': '0827198a456f77be', 'flushckid': '1', 'fromSearchBtn': '2', 'industries': '040', 'keyword': '数据分析', 'ckid': '0827198a456f77be', 'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw', 'd_sfrom': 'search_unknown', 'd_ckId': '41d615212789e37ab84f0440a67481f7', 'd_curPage': '0', 'd_pageSize': '40', 'd_headId': '41d615212789e37ab84f0440a67481f7'}, {'init': '-1', 'industryType': 'industry_01', 'headckid': '0827198a456f77be', 'flushckid': '1', 'fromSearchBtn': '2', 'industries': '420', 'keyword': '数据分析', 'ckid': '0827198a456f77be', 'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw', 'd_sfrom': 'search_unknown', 'd_ckId': '41d615212789e37ab84f0440a67481f7', 'd_curPage': '0', 'd_pageSize': '40', 'd_headId': '41d615212789e37ab84f0440a67481f7'}, {'init': '-1', 'industryType': 'industry_01', 'headckid': '0827198a456f77be', 'flushckid': '1', 'fromSearchBtn': '2', 'industries': '010', 'keyword': '数据分析', 'ckid': '0827198a456f77be', '

"python 原代碼以下不處理\ndf_qs = pd.DataFrame(list_query)\nprint (df_qs.nunique())\ndf_qs.head()\ndf_qs[['keyword','compTag']]\n"

In [70]:
# B-3-XX 整合后

列表暫存 = [] # list()
for q in df['query']: 
    字典暫存 = dict()
    for k,v in parse_qs(q).items(): # for 鍵,值 in 字典.items():
        字典暫存.update({k:v[0]})
    列表暫存.append(字典暫存)
列表暫存
# ----------------------------------------------
# 比較
# 列表暫存 = [{k:v[0] for k,v in parse_qs(q).items()} for q in df['query'] ]

[{'init': '-1',
  'industryType': 'industry_01',
  'headckid': '0827198a456f77be',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'industries': '040',
  'keyword': '数据分析',
  'ckid': '0827198a456f77be',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw',
  'd_sfrom': 'search_unknown',
  'd_ckId': '41d615212789e37ab84f0440a67481f7',
  'd_curPage': '0',
  'd_pageSize': '40',
  'd_headId': '41d615212789e37ab84f0440a67481f7'},
 {'init': '-1',
  'industryType': 'industry_01',
  'headckid': '0827198a456f77be',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'industries': '420',
  'keyword': '数据分析',
  'ckid': '0827198a456f77be',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw',
  'd_sfrom': 'search_unknown',
  'd_ckId': '41d615212789e37ab84f0440a67481f7',
  'd_curPage': '0',
  'd_pageSize': '40',
  'd_headId': '41d615212789e37ab84f0440a67481f7'},
 {'init': '-1',
  'industryType': 'industry_01',
  'headckid': '0827198a456f77be',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'indu

In [82]:
# 建构 参数模板  
参数_industries_数据分析 = {'互联网/电商': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['040'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '游戏产业': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['420'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '计算机软件': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['010'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, 'IT服务': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['030'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '电子/芯片/半导体': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['050'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '通信业': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['060'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '计算机/网络设备': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['020'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '房地产/建筑': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['080'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '规划/设计/装潢': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['100'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '房地产服务': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['090'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '银行': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['130'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '保险': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['140'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '基金/证券/投资': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['150'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '会计/审计': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['430'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '信托/担保/拍卖': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['500'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '快消品': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['190'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '批发零售': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['240'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '服装纺织': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['200'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '家具/家电': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['210'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '办公设备': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['220'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '奢侈品/收藏品': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['460'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '珠宝/玩具/工艺品': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['470'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '汽车/摩托车': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['350'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '机械/机电/重工': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['360'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '印刷/包装/造纸': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['180'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '原材料加工': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['370'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '仪器/电气/自动化': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['340'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '制药/生物工程': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['270'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '医疗/保健/美容': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['280'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '医疗器械': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['290'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '能源/水利': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['330'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '石油/化工': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['310'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '采掘/冶炼/矿产': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['320'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '环保': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['300'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '新能源': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['490'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '专业服务': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['120'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '中介服务': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['110'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '外包服务': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['440'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '检测/认证': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['450'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '餐饮/酒旅/服务': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['230'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '文体娱乐': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['260'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '租赁服务': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['510'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '广告/市场/会展': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['070'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '影视文化': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['170'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '教育培训': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['380'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '交通/物流/运输': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['250'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '贸易/进出口': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['160'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '航空/航天': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['480'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '政务/公共服务': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['390'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '农林牧渔': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['410'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}, '其他行业': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['1ec3bdba73fd7b1f'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['400'], 'keyword': ['数据分析'], 'ckid': ['1ec3bdba73fd7b1f'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['abc199670e4e54a28458aff73debc93d'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['abc199670e4e54a28458aff73debc93d']}}
# 上周C-1   多个页面准备测试1 互联网/电商
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
payload = 参数_industries_数据分析['互联网/电商']
r = session.get( url, params = payload)

# r.url

#  单一页面爬+解析
session = HTMLSession()

def requests_liepin( url, params):
    r = session.get( url , params = payload)

    # 先取特定元素, 精准打击其子后辈
    主要元素 = r.html.xpath( '//ul[@class="sojob-list"]/li')

    # 作为xpath字典，键为我要抓的牛肉名称，值为xpath
    dict_xpaths={ 
        'text': {
            'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
            '职称':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司地点': '//div[contains(@class,"job-info")]/p/a',
            '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        },
        'text_content': {
        },
        'href': {
            '链结':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        }
    }

    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
        return(暂存结果)

    def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in 主要元素]
        return(暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)


# 多个页面
url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_industries_数据分析.items():
    payload = v
    df = requests_liepin( url, params = payload)
    df = df.assign (数据分析 = k)    
    list_df.append(df)

df_all = pd.concat(list_df)
df_all

#   输出
df_all.to_excel("数据分析.xlsx", sheet_name="搜查结果")

# Pandas  基本能力

print (df_all.nunique())
df_all[['edu']].drop_duplicates()

df_all.groupby(['公司名称','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

edu         8
经验          6
薪水        312
时间         60
职称       1637
公司地点      441
公司名称     1166
链结       1935
公司URL    1166
数据分析       51
dtype: int64


,,职称
公司名称,edu,
米高蒲志国际(香港)有限公司,本科及以上,20
新华三集团,统招本科,18
立信会计师事务所,本科及以上,10
特变电工新能源,统招本科,10
上海蓝云实业发展有限公司,大专及以上,10
...,...,...
天津市柯林楼宇物业管理有限公司,大专及以上,1
天健会计师事务所(特殊普通合伙)江苏分所,统招本科,1
天仪研究院,硕士及以上,1


In [3]:
# A-1   单一页面
url = "https://www.liepin.com/zhaopin/?keyword=数据分析"
session = HTMLSession()
r = session.get( url )

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//ul[@class="sojob-list"]/li')

# 预期是一个元素的列表？
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]'))
#print (主要元素[0].xpath('//div[contains(@class,"job-info")]/h3/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="text-warning"]'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/time/@title'))
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a'))

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
        '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
        '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
        '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
        '职称':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司地点': '//div[contains(@class,"job-info")]/p/a',
        '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    },
    'text_content': {
    },
    'href': {
        '链结':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

[len(v) for k,v in 数据字典.items()]

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
数据 

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL
0,大专及以上,10年以上,25-35k·12薪,2020年04月26日,人防设计工程师,广州-黄埔区,广州嘉殷建设管理有限公司,https://www.liepin.com/job/1927791699.shtml,https://www.liepin.com/company/12195615/
1,大专及以上,3-5年,6-8k·12薪,2020年04月26日,测试工程师,西安-未央区,星展测控科技股份有限公司,https://www.liepin.com/job/1927789815.shtml,https://www.liepin.com/company/10158561/
2,学历不限,1-3年,4-6k·12薪,2020年04月26日,装配技工,西安-未央区,星展测控科技股份有限公司,https://www.liepin.com/job/1927789491.shtml,https://www.liepin.com/company/10158561/
3,大专及以上,1-3年,面议,2020年04月26日,客户经理（汽车融资租赁）,南昌,创富融资租赁(上海)有限公司,https://www.liepin.com/job/1927788671.shtml,https://www.liepin.com/company/7854759/
4,本科及以上,3-5年,16-25k·12薪,2020年04月26日,golang开发工程师,成都-高新区,成都零点能量信息技术有限责任公司,https://www.liepin.com/job/1927787679.shtml,https://www.liepin.com/company/9326766/
5,大专及以上,5-10年,15-20k·12薪,2020年04月26日,成本工程师（土建）,承德,承德天缘盛世房地产开发有限公司,https://www.liepin.com/job/1927785791.shtml,https://www.liepin.com/company/9143037/
6,大专及以上,3-5年,8-16k·12薪,2020年04月26日,渠道经理,苏州,苏州浩辰软件股份有限公司,https://www.liepin.com/job/1927784455.shtml,https://www.liepin.com/company/7965554/
7,大专及以上,5-10年,面议,2020年04月26日,区域销售经理,上海,上海迪罗电气有限公司,https://www.liepin.com/job/1927784195.shtml,https://www.liepin.com/company/7927458/
8,本科及以上,经验不限,面议,2020年04月26日,国际保险理赔总监,大连,维特奥健康,https://www.liepin.com/job/1927783939.shtml,https://www.liepin.com/company/8375949/
9,大专及以上,1-3年,5-7k·12薪,2020年04月26日,出纳,深圳-龙华区,北京斯雷康科技股份有限公司,https://www.liepin.com/job/1927783881.shtml,https://www.liepin.com/company/1408548/


In [4]:
url = "https://www.liepin.com/zhaopin/?keyword=数据分析"
session = HTMLSession()
r = session.get( url )

In [5]:
xpath_翻页a = '//div[@class="pagerbar"]/a' # 有disabled, current等href是javascript
xpath_翻页a = '//div[@class="pagerbar"]/a[starts-with(@href,"/zhaopin")]'
print (r.html.xpath(xpath_翻页a)) # 物件

href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
#print (href_列表)

文字_列表 = [x.text for x in r.html.xpath(xpath_翻页a)]
#print (文字_列表)

href_字典 = {x.text:x.xpath('//@href')[0]  for x in r.html.xpath(xpath_翻页a)}
print (href_字典)

[<Element 'a' href='/zhaopin/?init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=1bcfe9ed1345e5a9°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=5898ac1d58b8dd81d08b76d7f2606cdc&d_curPage=0&d_pageSize=40&d_headId=5898ac1d58b8dd81d08b76d7f2606cdc&curPage=1'>, <Element 'a' href='/zhaopin/?init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=1bcfe9ed1345e5a9°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=5898ac1d58b8dd81d08b76d7f2606cdc&d_curPage=0&d_pageSize=40&d_headId=5898ac1d58b8dd81d08b76d7f2606cdc&curPage=2'>, <Element 'a' href='/zhaopin/?init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=1bcfe9ed1345e5a9°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=5898ac1d58b8dd81d08b76d7f2606cdc&d_curPage=0&d_p

In [6]:
xpath_翻页a = '//div[@class="pagerbar"]/a' # 有disabled, current等href是javascript
xpath_翻页a = '//div[@class="pagerbar"]/a[starts-with(@href,"/zhaopin")]'
print (r.html.xpath(xpath_翻页a)) # 物件

href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
#print (href_列表)

文字_列表 = [x.text for x in r.html.xpath(xpath_翻页a)]
#print (文字_列表)

href_字典 = {x.text:x.xpath('//@href')[0]  for x in r.html.xpath(xpath_翻页a)}
print (href_字典)

[<Element 'a' href='/zhaopin/?init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=1bcfe9ed1345e5a9°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=5898ac1d58b8dd81d08b76d7f2606cdc&d_curPage=0&d_pageSize=40&d_headId=5898ac1d58b8dd81d08b76d7f2606cdc&curPage=1'>, <Element 'a' href='/zhaopin/?init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=1bcfe9ed1345e5a9°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=5898ac1d58b8dd81d08b76d7f2606cdc&d_curPage=0&d_pageSize=40&d_headId=5898ac1d58b8dd81d08b76d7f2606cdc&curPage=2'>, <Element 'a' href='/zhaopin/?init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=1bcfe9ed1345e5a9°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=5898ac1d58b8dd81d08b76d7f2606cdc&d_curPage=0&d_p

In [7]:
from urllib.parse import urlparse, parse_qs
import pandas as pd
from IPython.display import display, HTML

# 总体目标：输入 href_列表, 建构出参数字典

# urlparse 解析后丢入数据框
df = pd.DataFrame([ urlparse(x) for x in href_列表])
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])

display(df)
print(df.nunique())
display(df_qs)
print(df_qs.nunique())

df_qs.curPage
df_qs = df_qs.assign (curPage_int=df_qs.curPage.astype(int)) #

,scheme,netloc,path,params,query,fragment
0,,,/zhaopin/,,init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBt...,
1,,,/zhaopin/,,init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBt...,
2,,,/zhaopin/,,init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBt...,
3,,,/zhaopin/,,init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBt...,
4,,,/zhaopin/,,init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBt...,
5,,,/zhaopin/,,init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBt...,


scheme      1
netloc      1
path        1
params      1
query       5
fragment    1
dtype: int64


,init,headckid,fromSearchBtn,keyword,ckid,siTag,d_sfrom,d_ckId,d_curPage,d_pageSize,d_headId,curPage
0,-1,1bcfe9ed1345e5a9,2,数据分析,1bcfe9ed1345e5a9°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,5898ac1d58b8dd81d08b76d7f2606cdc,0,40,5898ac1d58b8dd81d08b76d7f2606cdc,1
1,-1,1bcfe9ed1345e5a9,2,数据分析,1bcfe9ed1345e5a9°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,5898ac1d58b8dd81d08b76d7f2606cdc,0,40,5898ac1d58b8dd81d08b76d7f2606cdc,2
2,-1,1bcfe9ed1345e5a9,2,数据分析,1bcfe9ed1345e5a9°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,5898ac1d58b8dd81d08b76d7f2606cdc,0,40,5898ac1d58b8dd81d08b76d7f2606cdc,3
3,-1,1bcfe9ed1345e5a9,2,数据分析,1bcfe9ed1345e5a9°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,5898ac1d58b8dd81d08b76d7f2606cdc,0,40,5898ac1d58b8dd81d08b76d7f2606cdc,4
4,-1,1bcfe9ed1345e5a9,2,数据分析,1bcfe9ed1345e5a9°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,5898ac1d58b8dd81d08b76d7f2606cdc,0,40,5898ac1d58b8dd81d08b76d7f2606cdc,1
5,-1,1bcfe9ed1345e5a9,2,数据分析,1bcfe9ed1345e5a9°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,5898ac1d58b8dd81d08b76d7f2606cdc,0,40,5898ac1d58b8dd81d08b76d7f2606cdc,9


init             1
headckid         1
fromSearchBtn    1
keyword          1
ckid             1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
curPage          5
dtype: int64


In [8]:
def parse_url_qs_for_curPage (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# 取一例做模板
参数模板 = parse_url_qs_for_curPage(href_列表[0])
print (参数模板)

print (href_字典)

{'init': ['-1'], 'headckid': ['1bcfe9ed1345e5a9'], 'fromSearchBtn': ['2'], 'keyword': ['数据分析'], 'ckid': ['1bcfe9ed1345e5a9°radeFlag=0'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['5898ac1d58b8dd81d08b76d7f2606cdc'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['5898ac1d58b8dd81d08b76d7f2606cdc'], 'curPage': ['1']}
{'2': '/zhaopin/?init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=1bcfe9ed1345e5a9°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=5898ac1d58b8dd81d08b76d7f2606cdc&d_curPage=0&d_pageSize=40&d_headId=5898ac1d58b8dd81d08b76d7f2606cdc&curPage=1', '3': '/zhaopin/?init=-1&headckid=1bcfe9ed1345e5a9&fromSearchBtn=2&keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&ckid=1bcfe9ed1345e5a9°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=5898ac1d58b8dd81d08b76d7f2606cdc&d_curPage=0

In [9]:
def 参数模板生成(keyword, curPage):
    参数 = 参数模板.copy()
    参数['curPage'] = curPage
    参数['keyword'] = keyword
    return (参数)

参数_keyword_数据分析_curPage = { 
    i:参数模板生成(curPage = [i], \
                  keyword = ['shu']) \
    for i,v in href_字典.items()\
    }

# print(参数_keyword_用户体验_curPage) # 只生成本页有的额外翻页URL, 并没有推估到&curPage=9,也没有这页

print (df_qs.curPage_int.min()) # 最小值只有1
print (df_qs.curPage_int.max()) # 最大值只有9

# 应该是 0 (本页)....9(最大值)

参数_keyword_用户体验_curPage = { 
    i:参数模板生成(curPage = [i], \
                  keyword = ['数据分析']) \
    for i in range(0,df_qs.curPage_int.max()+1)\
    }
参数_keyword_用户体验_curPage

1
9


{0: {'init': ['-1'],
  'headckid': ['1bcfe9ed1345e5a9'],
  'fromSearchBtn': ['2'],
  'keyword': ['数据分析'],
  'ckid': ['1bcfe9ed1345e5a9°radeFlag=0'],
  'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
  'd_sfrom': ['search_unknown'],
  'd_ckId': ['5898ac1d58b8dd81d08b76d7f2606cdc'],
  'd_curPage': ['0'],
  'd_pageSize': ['40'],
  'd_headId': ['5898ac1d58b8dd81d08b76d7f2606cdc'],
  'curPage': [0]},
 1: {'init': ['-1'],
  'headckid': ['1bcfe9ed1345e5a9'],
  'fromSearchBtn': ['2'],
  'keyword': ['数据分析'],
  'ckid': ['1bcfe9ed1345e5a9°radeFlag=0'],
  'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
  'd_sfrom': ['search_unknown'],
  'd_ckId': ['5898ac1d58b8dd81d08b76d7f2606cdc'],
  'd_curPage': ['0'],
  'd_pageSize': ['40'],
  'd_headId': ['5898ac1d58b8dd81d08b76d7f2606cdc'],
  'curPage': [1]},
 2: {'init': ['-1'],
  'headckid': ['1bcfe9ed1345e5a9'],
  'fromSearchBtn': ['2'],
  'keyword': ['数据分析'],
  'ckid': ['1bcfe9ed1345e5a9°radeFlag=0'],
  'siTag': ['1B2M2Y8AsgTpgAmY

In [10]:
session = HTMLSession()

def requests_liepin( url, params):
    r = session.get( url , params = payload)

    # 先取特定元素, 精准打击其子后辈
    主要元素 = r.html.xpath( '//ul[@class="sojob-list"]/li')

    # 作为xpath字典，键为我要抓的牛肉名称，值为xpath
    dict_xpaths={ 
        'text': {
            'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
            '职称':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司地点': '//div[contains(@class,"job-info")]/p/a',
            '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        },
        'text_content': {
        },
        'href': {
            '链结':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        }
    }

    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
        return(暂存结果)

    def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in 主要元素]
        return(暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)

In [17]:
%%time
time.sleep(3+4*random())

Wall time: 5.04 s


In [18]:
# B-2 多个页面，但放慢脚步 time.sleep
%%time

import time
from random import random

url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_keyword_数据分析_curPage.items():
    payload = v
    df = requests_liepin( url, params = payload)
    time.sleep(3+4*random())  #放慢脚步 3-7秒, 平均约5秒
    df = df.assign (curPage = k)  # 区分  curPage
    list_df.append(df)

df_all = pd.concat(list_df).reset_index()
df_all.index.name = '序'

# 上周C-4   输出
# df_all.to_excel("20春_Web数据挖掘_week04_liepin_翻页.xlsx",\
                #sheet_name="用户体验")


UsageError: Line magic function `%%time` not found.


In [20]:
%%time
# C-1 多个页面+多个关键词
import time
from random import random

url = "https://www.liepin.com/zhaopin/"
xpath_翻页a = '//div[@class="pagerbar"]/a[starts-with(@href,"/zhaopin")]'

keywords = ['数据分析','数据挖掘','用户研究']
list_df = list()

## 第一页试探有多长的页面
for key in keywords:
    payload = 参数模板生成(keyword=[key], curPage=['0'])
    df = requests_liepin( url, params = payload)
    href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
    df = pd.DataFrame([ urlparse(x) for x in href_列表])
    df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
    df_qs = df_qs.assign (curPage_int=df_qs.curPage.astype(int)) # 变成整数
    长度 = df_qs.curPage_int.max()+1
    参数_keyword_X_curPage = { 
        i:参数模板生成(curPage = [i], \
                      keyword = [key]) \
        for i in range(0,长度)\
        }
    #print (参数_keyword_X_curPage)
    print (key,长度)
    
    for k,v in 参数_keyword_X_curPage.items():
        payload = v
        df = requests_liepin( url, params = payload)
        time.sleep(3+4*random())  #放慢脚步 3-7秒, 平均约5秒
        ## 备份
        df.to_csv("20春_Web数据挖掘_week04_liepin_{key}_{k}.tsv"\
                  .format(key=key, k=k), sep="\t", encoding="utf8")
        
        df = df.assign (keyword = key)  # 区分  keyword    
        df = df.assign (curPage = k)  # 区分  curPage    
        list_df.append(df)
        
df_all = pd.concat(list_df).reset_index()
df_all.index.name = '序'

df_all.to_excel("数据分析行业情况.xlsx",\
                sheet_name="_".join(keywords))
# 预估时间: 4*5秒*10 =200
# 预估数量: 4*40*10 =1600

数据分析 10
数据挖掘 10
用户研究 10
Wall time: 2min 58s
